In [32]:
import pandas as pd
import numpy as np

import nltk
from nltk.stem import PorterStemmer
import re

from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB


import urllib.request
import webbrowser

In [33]:
from nltk.corpus import stopwords
quit_words = ['quit', 'bye', 'thanks', 'exit', 'thankyou', 'thank you', 'thanku']

In [34]:
# Common functions definition

def cleanText(s1):
    ps = PorterStemmer()
    text = s1.lower()
    text = re.sub(r'[0-9]*','',text)
    text = re.sub(r'\s\s+',' ',text)
    text = [ps.stem(word) for word in nltk.word_tokenize(text) if word not in stopwords.words('english')]
    text = ' '.join(text)
    return text

# After prediction convert tag to response
def getResponse(tag):
    try:
        if(resp_dict[tag]):
            response = resp_dict[tag]
            return response
        else:
            print('Response not found.. Please try another query.')
    except:
        print('Response not found for:', tag)
        print('I will update my knowledge soon :)')

def process_response(response):
    if(response[0:4]=='Link'):
        url = response[5:]
#        webbrowser.open(url)
        print("url to open is {}".format(url))
    else:
        print(response)

In [35]:
df = pd.read_csv("Corpus_1.csv")
df.columns = ['query', 'response', 'tag']
df = df.dropna()

In [36]:
df.head(2)

,query,response,tag
0,Hello,Hello! how can I help you ?,hello
1,Linear Regression formula,Link:https://en.wikipedia.org/wiki/Linear_regr...,Linear regression


In [37]:
df['transformed_query'] = df['query'].apply(cleanText)

In [38]:
# encoding the target column
le = LabelEncoder()
df['tag_encoded'] = le.fit_transform(df['tag'])

In [39]:
#df_ml = df[['transformed_query', 'tag_encoded']]

In [40]:
# Converting with bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(df['transformed_query']).toarray()

y = df['tag_encoded']

In [41]:
# Creating dataframe to visualize data
temp_name = cv.get_feature_names()
temp = pd.DataFrame(X, columns=temp_name)

In [42]:
# Create dictionary for tag to response
temp = df[['tag_encoded', 'response']]
temp = temp.groupby(['tag_encoded']).max()
resp_dict = temp.to_dict()
resp_dict = resp_dict['response']

In [43]:
# Fitting naive bayes algorithm
mnb = MultinomialNB()
model = mnb.fit(X, y)

In [44]:
ss = 'library used for algorithms'
query_df = process_input(ss)
X_test     = cv.transform(query_df['query']).toarray()
prediction = model.predict(X_test)
print(prediction)
resp       = getResponse(prediction[0])
print(resp)
process_response(resp)


[10]
Sklearn
Sklearn


In [45]:
# Predict using model
def process_input(s1):
#    s1 = 'What is linear regression?'
    s1 = cleanText(s1)
    l1=[]
    l1.append(s1)
    ldf = pd.DataFrame(l1, columns= ['query'])
    return ldf
    

### Start of prediction flow

In [46]:
# Get input from user
inp_text=''
while(inp_text != 'quit'):
    inp_text = input('Please enter your question: ')
    if (inp_text in quit_words):
        inp_text = 'quit'
    if(inp_text != 'quit'):
        query_df   = process_input(inp_text)
#         print(query_df)
        X_test     = cv.transform(query_df['query']).toarray()
        prediction = model.predict(X_test)
        resp       = getResponse(prediction[0])
        process_response(resp)
    else:
        print("Thanks for your visit")

Please enter your question: library used in machine learning
Sklearn
Please enter your question: bye
Thanks for your visit
